# Text Main Streaming

> This module contains the main Python class for data control for streaming data: `TextDataControllerStreaming`

- skip_showdoc: true
- skip_exec: true

In [ ]:
#| default_exp text_main_streaming

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,MultiLabelBinarizer
from datasets import DatasetDict,Dataset,IterableDataset,load_dataset,concatenate_datasets,Value
from pathlib import Path
from that_nlp_library.utils import *
from that_nlp_library.text_main import tokenize_function,concat_metadatas
from functools import partial
import warnings

In [ ]:
import pandas as pd
import numpy as np
from that_nlp_library.text_transformation import *
from that_nlp_library.text_augmentation import *
from importlib.machinery import SourceFileLoader
import os

## Class TextDataControllerStreaming

In [ ]:
#| export
class TextDataControllerStreaming():
    def __init__(self,
                 inp, # HuggingFainpce Dataset or DatasetDict
                 main_text:str, # Name of the main text column
                 label_names=None, # Names of the label (dependent variable) columns
                 class_names_predefined=None, # List of names associated with the labels (same index order)
                 filter_dict={}, # A dictionary: {feature: filtering_function_based_on_the_feature}
                 metadatas=[], # Names of the metadata columns
                 process_metas=True, # Whether to do simple text processing on the chosen metadatas
                 content_transformations=[], # A list of text transformations
                 content_augmentations=[], # A list of text augmentations
                 seed=None, # Random seed
                 batch_size=100, # CPU batch size
                 num_proc=1, # Number of process for multiprocessing. This will be applied on non-streamed validation set
                 cols_to_keep=None, # Columns to keep after all processings
                 verbose=True, # Whether to print processing information
                ):
            
        self.main_text = main_text
        self.metadatas = val2iterable(metadatas)
        self.process_metas = process_metas
        self.label_names = val2iterable(label_names) if label_names is not None else None
        self.label_lists = class_names_predefined
        self.filter_dict = filter_dict
        self.content_tfms = val2iterable(content_transformations)
        self.aug_tfms = val2iterable(content_augmentations)
        self.seed = seed
        self.is_batched = batch_size>1
        self.batch_size = batch_size
        self.num_proc = num_proc
        self.cols_to_keep = cols_to_keep
        self.verbose = verbose
        self.main_ddict=DatasetDict()
        self.verboseprint = print if verbose else lambda *a, **k: None
        
        if hasattr(inp,'keys'): # is datasetdict
            if 'train' not in inp.keys(): 
                raise ValueError('The given DatasetDict has no "train" split')
            else:
                self.main_ddict['train'] = inp['train']
            val_key = list(set(inp.keys()) & set(['val','validation','valid']))
            if len(val_key)>1: raise ValueError('Your DatasetDict has more than 1 validation split')
            if len(val_key)==1:
                self.main_ddict['validation'] = inp[val_key[0]]
        else: # is dataset
            self.main_ddict['train'] = inp
        
        
        
        is_streamed=isinstance(self.main_ddict['train'],IterableDataset)
        if not is_streamed: raise Exception('This Text Data Controller only handles streamed dataset')
        
        self.all_cols = get_dset_col_names(self.main_ddict['train'])
        if is_streamed and self.label_names is not None and self.label_lists is None:
            raise ValueError('All class labels must be provided when streaming')
            
        self._processed_call=False
        
        self._determine_multihead_multilabel()
        
            
    @classmethod
    def from_pickle(cls,
                    fname, # Name of the pickle file
                    parent='pickle_files' # Parent folder
                   ):
        return load_pickle(fname,parent=parent)
    
    def set_verbose(self,verbose):
        self.verbose = verbose
        self.verboseprint = print if verbose else lambda *a, **k: None
        
    def _determine_multihead_multilabel(self):
        self.is_multilabel=False
        self.is_multihead=False
        if self.label_names is None: return
        
        if len(self.label_names)>1:
            self.is_multihead=True
        # get label of first row
        first_label = next(iter(self.main_ddict['train']))[self.label_names[0]]
        if isinstance(first_label,(list,set,tuple)):
            # This is multi-label. Ignore self.label_names[1:]
            self.label_names = [self.label_names[0]]
            self.is_multihead=False
            self.is_multilabel=True
                     
    
    def save_as_pickles(self,
                        fname, # Name of the pickle file
                        parent='pickle_files', # Parent folder
                        drop_attributes=False # Whether to drop large-size attributes
                       ):
        if drop_attributes:
            if hasattr(self, 'main_ddict'):
                del self.main_ddict
            if hasattr(self, 'ddict_rest'):
                del self.ddict_rest
            if hasattr(self, 'aug_tfms'):
                del self.aug_tfms
        save_to_pickle(self,fname,parent=parent)
    
                             
    def _create_label_mapping_func(self,encoder_classes):
        if self.is_multihead:
            label2idxs = [{v:i for i,v in enumerate(l_classes)} for l_classes in encoder_classes]
            _func = lambda inp: {'label': [[label2idxs[i][v] for i,v in enumerate(vs)] for vs in zip(*[inp[l] for l in self.label_names])] \
                                    if self.is_batched else [label2idxs[i][v] for i,v in enumerate([inp[l] for l in self.label_names])]
                              }
            
        else:
            label2idx = {v:i for i,v in enumerate(encoder_classes[0])}
            _func = partial(lambda_map_batch,
                           feature=self.label_names[0],
                           func=lambda x: label2idx[x],
                           output_feature='label',
                           is_batched=self.is_batched)
        return _func
        
    def _encode_labels(self):
        if self.label_names is None: return
        print_msg('Label Encoding',verbose=self.verbose)
        
        if not isinstance(self.label_lists[0],list):
            self.label_lists = [self.label_lists]
                    
        encoder_classes=[]
        if not self.is_multilabel:
            for idx,l in enumerate(self.label_names):
                l_classes = sorted(list(self.label_lists[idx]))
                encoder_classes.append(l_classes)
            _func = self._create_label_mapping_func(encoder_classes)
            self.main_ddict['train'] = hf_map_dset(self.main_ddict['train'],_func,self.is_batched,self.batch_size,self.num_proc)
            if 'validation' in self.main_ddict.keys():
                self.main_ddict['validation'] = hf_map_dset(self.main_ddict['validation'],_func,self.is_batched,self.batch_size,self.num_proc)
                    
        else:
            # For MultiLabel, we transform the label itself to one-hot (or actually, few-hot)
            l_classes = sorted(list(self.label_lists[0]))   
            encoder_classes.append(l_classes)
            
            l_encoder = MultiLabelBinarizer(classes=encoder_classes[0])
            _ = l_encoder.fit(None)
            _func = partial(lambda_map_batch,
                            feature=self.label_names[0],
                            func=lambda x: l_encoder.transform(x),
                            output_feature='label',
                            is_batched=self.is_batched,
                            is_func_batched=True)
            self.main_ddict['train'] = hf_map_dset(self.main_ddict['train'],_func,self.is_batched,self.batch_size,self.num_proc)
            if 'validation' in self.main_ddict.keys():
                self.main_ddict['validation'] = hf_map_dset(self.main_ddict['validation'],_func,self.is_batched,self.batch_size,self.num_proc)
            
        self.label_lists = encoder_classes
        self.verboseprint('Done')
        
    def _process_metadatas(self,dtrain):
        if len(self.metadatas)>0:
            map_func = partial(concat_metadatas,
                               main_text=self.main_text,
                               metadatas=self.metadatas,
                               process_metas=self.process_metas,
                               is_batched=self.is_batched)
            dtrain = hf_map_dset(dtrain,map_func,self.is_batched,self.batch_size,self.num_proc)
        return dtrain
            
            
    def _simplify_ddict(self):
        print_msg('Dropping unused features',20,verbose=self.verbose)
        if self.cols_to_keep is None:
            self.cols_to_keep= [self.main_text] + self.metadatas
            if self.label_names is not None: self.cols_to_keep+=self.label_names
        cols_to_remove = set(self.all_cols) - set(self.cols_to_keep)
        self.main_ddict['train']=self.main_ddict['train'].remove_columns(list(cols_to_remove))
        if 'validation' in self.main_ddict.keys():
            self.main_ddict['validation']=self.main_ddict['validation'].remove_columns(list(cols_to_remove))
        self.verboseprint('Done')

    def _do_filtering(self,dtrain):
        if len(self.filter_dict):
            col_names = get_dset_col_names(dtrain)
            for f,tfm in self.filter_dict.items():
                if f in col_names:
                    _func = partial(lambda_batch,
                                    feature=f,
                                    func=tfm,
                                    is_batched=self.is_batched)
                    dtrain = hf_filter_dset(dtrain,_func,self.is_batched,self.batch_size,self.num_proc)
        return dtrain
        

    def _do_transformation_tokenization(self,dtrain,tokenizer,max_length,):
        tok_func = partial(tokenize_function,tok=tokenizer,max_length=max_length)
        if len(self.content_tfms):            
            for tfm in self.content_tfms:
                _func = partial(lambda_map_batch,
                                feature=self.main_text,
                                func=tfm,
                                is_batched=self.is_batched)
                dtrain = hf_map_dset(dtrain,_func,self.is_batched,self.batch_size,self.num_proc)
        
        _func = partial(lambda_map_batch,
                        feature=self.main_text,
                        func=tok_func,
                        output_feature=None,
                        is_batched=self.is_batched)
        dtrain = hf_map_dset(dtrain,_func,self.is_batched,self.batch_size,self.num_proc)
            
        return dtrain 
 
    def _do_transformation_augmentation_tokenization(self,tokenizer,max_length):
        tok_func = partial(tokenize_function,tok=tokenizer,max_length=max_length)
        all_tfms = self.content_tfms + self.aug_tfms
        all_tfms = partial(func_all,functions=all_tfms) if len(all_tfms) else None
        seed_everything(self.seed)
           
        self.main_ddict['train'] = IterableDataset.from_generator(aug_and_tok_stream_generator,
                                                   gen_kwargs={'dset': self.main_ddict['train'],
                                                               'text_name':self.main_text,
                                                               'tok_func':tok_func,
                                                               'func': all_tfms
                                                              }
                                                                 )
        
        
    def process_and_tokenize(self,
                             tokenizer, # Tokenizer (preferably from HuggingFace)
                             max_length=None, # pad to model's allowed max length (default is max_sequence_length)
                            ):
        if self._processed_call:
            warnings.warn('Your dataset has already been processed. Returning the previous processed DatasetDict...')
            return self.main_ddict
        
        self.tokenizer = tokenizer
        self.max_length = max_length
                             
        # Filtering
        print_msg('Data Filtering',20,verbose=self.verbose)
        for k in self.main_ddict.keys():   
            self.main_ddict[k] = self._do_filtering(self.main_ddict[k])
        self.verboseprint('Done')

        
        # Process metadatas
        print_msg('Metadata Simple Processing & Concatenating to Main Content',verbose=self.verbose)
        for k in self.main_ddict.keys():   
            self.main_ddict[k] = self._process_metadatas(self.main_ddict[k])
        self.verboseprint('Done')
        
        # Process labels
        self._encode_labels()

        
        # Dropping unused columns
        self._simplify_ddict()

        
        # Content transformation + tokenization for validation
        if 'validation' in self.main_ddict.keys():
            print_msg('Performing content transformation and tokenization on validation set',verbose=self.verbose)
            self.main_ddict['validation'] = self._do_transformation_tokenization(self.main_ddict['validation'],
                                                                                 tokenizer,
                                                                                 max_length,
                                                                                )
            self.verboseprint('Done')
 
        # Content transformation + augmentation + tokenization for train
        print_msg('Creating a generator for content transformation, augmentation and tokenization on train set',verbose=self.verbose)
        self._do_transformation_augmentation_tokenization(tokenizer,max_length)
        self.verboseprint('Done')
        self._processed_call=True
    
        
    
    def set_data_collator(self,data_collator):
        self.data_collator = data_collator
        
    
    def prepare_test_dataset_from_csv(self,
                                      file_path, # path to csv file
                                      do_filtering=False # whether to perform data filtering on this test set
                                     ):
        file_path = Path(file_path)
        ds = load_dataset(str(file_path.parent),
                          data_files=file_path.name,
                          split='train')
        return self.prepare_test_dataset(ds,do_filtering)
    
    def prepare_test_dataset_from_df(self,
                                     df, # Pandas Dataframe
                                     validate=True, # whether to perform input data validation
                                     do_filtering=False # whether to perform data filtering on this test set 
                                    ):
        if validate:
            check_input_validation(df)
        ds = Dataset.from_pandas(df)
        return self.prepare_test_dataset(ds,do_filtering)
    
    def prepare_test_dataset_from_raws(self,
                                       content, # Either a single sentence, list of sentence or a dictionary with keys are metadata columns and values are list
                                      ):
        if len(self.metadatas)!=0 and not isinstance(content,dict):
            raise ValueError(f'There is/are metadatas in the preprocessing step. Please include a dictionary including these keys for metadatas: {self.metadatas}, and texture content: {self.main_text}')
            
        _dic = {self.main_text:[content]} if isinstance(content,str) else content
        for k in _dic.keys():
            _dic[k] = val2iterable(_dic[k])
        
        test_dict = Dataset.from_dict(_dic)
        return self.prepare_test_dataset(test_dict,do_filtering=False)
    
    def prepare_test_dataset(self,
                             test_dset, # The HuggingFace Dataset as Test set
                             do_filtering=False, # whether to perform data filtering on this test set
                            ):
        test_cols = set(get_dset_col_names(test_dset))
        if self.label_names is None: # no label (i.e. regression)
            label_names_set = set([])
        else:
            label_names_set = set(self.label_names)
        test_cols = test_cols - label_names_set
        missing_cols = set(self.cols_to_keep) - label_names_set - test_cols
        if len(missing_cols):
            raise ValueError(f'Test set does not have these columns required for preprocessings: {missing_cols}')
            
        print_msg('Start Test Set Transformation',20,verbose=self.verbose)

        # Filtering
        if do_filtering:
            print_msg('Data Filtering',20,verbose=self.verbose)
            test_dset = self._do_filtering(test_dset)
            self.verboseprint('Done')
        
        # Process metadatas
        print_msg('Metadata Simple Processing & Concatenating to Main Content',verbose=self.verbose)    
        test_dset = self._process_metadatas(test_dset)
        self.verboseprint('Done')
        
        # Drop unused columns
        print_msg('Dropping unused features',20,verbose=self.verbose)
        cols_to_remove = test_cols - set(self.cols_to_keep)
        test_dset=test_dset.remove_columns(list(cols_to_remove))
        self.verboseprint('Done')
        
        
        # Content transformation and tokenization
        print_msg('Performing content transformation and tokenization on test set',verbose=self.verbose)
        test_dset = self._do_transformation_tokenization(test_dset,self.tokenizer,self.max_length)
        self.verboseprint('Done')
        
        return test_dset


In [ ]:
show_doc(TextDataControllerStreaming)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L662){target="_blank" style="float:right; font-size:smaller"}

### TextDataControllerStreaming

>      TextDataControllerStreaming (inp, main_text:str, label_names=None,
>                                   class_names_predefined=None, filter_dict={},
>                                   metadatas=[], process_metas=True,
>                                   content_transformations=[],
>                                   content_augmentations=[], seed=None,
>                                   batch_size=100, num_proc=1,
>                                   cols_to_keep=None, verbose=True)

Initialize self.  See help(type(self)) for accurate signature.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| inp |  |  | HuggingFainpce Dataset or DatasetDict |
| main_text | str |  | Name of the main text column |
| label_names | NoneType | None | Names of the label (dependent variable) columns |
| class_names_predefined | NoneType | None | List of names associated with the labels (same index order) |
| filter_dict | dict | {} | A dictionary: {feature: filtering_function_based_on_the_feature} |
| metadatas | list | [] | Names of the metadata columns |
| process_metas | bool | True | Whether to do simple text processing on the chosen metadatas |
| content_transformations | list | [] | A list of text transformations |
| content_augmentations | list | [] | A list of text augmentations |
| seed | NoneType | None | Random seed |
| batch_size | int | 100 | CPU batch size |
| num_proc | int | 1 | Number of process for multiprocessing. This will be applied on non-streamed validation set |
| cols_to_keep | NoneType | None | Columns to keep after all processings |
| verbose | bool | True | Whether to print processing information |

In [ ]:
show_doc(TextDataController.process_and_tokenize)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L570){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.process_and_tokenize

>      TextDataController.process_and_tokenize (tokenizer, max_length=None,
>                                               trn_size=None, shuffle_trn=True)

This will perform `do_all_processing` then `do_tokenization`

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| tokenizer |  |  | Tokenizer (preferably from HuggingFace) |
| max_length | NoneType | None | pad to model's allowed max length (default is max_sequence_length) |
| trn_size | NoneType | None | The number of training data to be tokenized |
| shuffle_trn | bool | True | To shuffle the train set before tokenization |

## Streaming Capability

The majority of streaming capability of `TextDataControllerStreaming` is adapted from [HuggingFace's stream](https://huggingface.co/docs/datasets/stream)

Streaming is a method to let you work with data without having it in your hard drive. This is especially helpful when the dataset size exceeds the amount of disk space you have on your machine.

Here are a few things to be aware of when using `TextDataControllerStreaming` streaming functionality (versus `TextDataController`)

- The list of label names must be available beforehand
- To avoid out-of-memory error, reduce batch_size argument.
- There will be validation split functionality. If you want to include a validation set, provide a 'validation' split in your HuggingFace DatasetDict beforehand
- There's no upsampling, and there's no shuffling the training set
	

**To stream, you must provide a streamed HuggingFace dataset.**

Let's repeat few examples above, but with a streaming dataset

### Filtering + Metadatas + Content Transformation + Content Augmentation (for Single Head)

In [ ]:
from underthesea import text_normalize
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw


In [ ]:
def nlp_aug_stochastic(x,aug=None,p=0.5):
    if not isinstance(x,list): 
        if random.random()<p: return aug.augment(x)[0]
        return x
    news=[]
    originals=[]
    for _x in x:
        if random.random()<p: news.append(_x)
        else: originals.append(_x)
    # only perform augmentation when needed
    if len(news): news = aug.augment(news)
    return news+originals

In [ ]:
aug2 = naw.ContextualWordEmbsAug(model_path='roberta-base', 
                                device='cuda:0', # if you don't have gpu, change to 'cpu'
                                action="substitute",
                                top_k=10,
                               aug_p=0.07)

contextual_aug_func = partial(nlp_aug_stochastic,aug=aug2,p=0.5)

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
ddict_with_val = dset.train_test_split(test_size=0.1)
ddict_with_val['validation'] = ddict_with_val['test']
ddict_with_val['train'] = ddict_with_val['train'].to_iterable_dataset()
del ddict_with_val['test']

tdc = TextDataControllerStreaming(ddict_with_val,
                                 main_text='Review Text',
                                 label_names='Department Name',
                                 class_names_predefined=['Bottoms', 'Dresses', 'Intimate', 'Jackets', 'Tops', 'Trend'],
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 metadatas=['Title','Division Name'],
                                 content_transformations=[text_normalize,str.lower],
                                 content_augmentations= contextual_aug_func, 
                                 process_metas=True,
                                 batch_size=1000,
                                 num_proc=4,
                                 seed=42
                                )
tdc.process_and_tokenize(tokenizer,max_length=512)

-------------------- Data Filtering --------------------
Done
----- Metadata Simple Processing & Concatenating to Main Content -----
Done
----- Label Encoding -----
Done
-------------------- Dropping unused features --------------------
Done
----- Performing content transformation and tokenization on validation set -----


Map (num_proc=4):   0%|          | 0/2253 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2253 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2253 [00:00<?, ? examples/s]

Done
----- Creating a generator for content transformation, augmentation and tokenization on train set -----
Done


In [ ]:
tdc.main_ddict

DatasetDict({
    train: <datasets.iterable_dataset.IterableDataset object>
    validation: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2253
    })
})

In [ ]:
%%time
for i,v in enumerate(tdc.main_ddict['train']):
    if i%100==0:
        print(i)
    if i==1000:
        break
    pass
    

0
100
200
300
400
500
600
700
800
900
1000
CPU times: user 22.3 s, sys: 3.71 ms, total: 22.3 s
Wall time: 22.3 s


In [ ]:
for i,v in enumerate(tdc.main_ddict['train']):
    if i==10:break
    print(f"Text: {v['Review Text']}\nLabel: {v['Department Name']} => {v['label']}")
    print('-'*10)

Text: general petite . beautiful top , worth the necessary tailoring . the beautiful bold print drew me to this top and it did not disappoint upon receipt . however , the bottom ruffle belled so far out on each side that it was laughable ! the actual fit is nothing like the picture ; clearly the model's arms are placed in front of all the extra fabric to hold the ruffle back . however , the fabric is beautiful , the fit was perfect ( size 2 , 5 ' 4 " , 106 lbs . ) , the quality is great and i love the print so i decided to take it to my tailor to " sew away " the " wings " on both si
Label: Tops => 4
----------
Text: general . not as short on me ( petite ) . i ordered the xxs p as this dress is not a fitted dress , and that was the right size for me . only thing is the length is a bit linger still 9 lower on calf for me ) , the straps are almost tight , so i would say the dress is a reversed taper shape . color is beautiful , i ordered green as the other color ( plum ) doesn't have pet

In [ ]:
for i in range(5):
    print(f"Text: {tdc.main_ddict['validation']['Review Text'][i]}")
    print(f"Label: {tdc.main_ddict['validation']['Department Name'][i]} => {tdc.main_ddict['validation']['label'][i]}")
    print('-'*10)

Text: general . soft , feminine and fun pockets ! . i love this tunic . purchased the dark orange in medium ( i am 5 ' 9 and 140 lbs ) . tried the small and almost kept it but i felt seams around my arm pits a tad , so went with the medium and glad i did - this top should be comfortable . feels very fall and perfect for casual get-togethers and running around town . only comment is that it is rayon ... and for me anyway rayon doesn't wash too well - so we shall see how this one fairs .
Label: Tops => 4
----------
Text: general petite . a new staple ! . tried these on out of sheer curiosity -- i've got a long torso & was pleasantly surprised how flattering they are ! they manage to look flowing & sleek without shortening the legs . took a size 6 with my 27 " waist , 37 " hips . it's a bit of a generous fit , especially around the waist , but they're extremely comfortable & have room to tuck tops into . i have the cowled sweater tank in gray & it looks fantastic over these ! couldn't res

### Filtering + Metadatas + Content Transformation + Content Augmentation (for Multi Head)

In [ ]:
aug2 = naw.ContextualWordEmbsAug(model_path='roberta-base', 
                                device='cuda:0', # if you don't have gpu, change to 'cpu'
                                action="substitute",
                                top_k=10,
                               aug_p=0.07)

contextual_aug_func = partial(nlp_aug_stochastic,aug=aug2,p=0.5)

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
ddict_with_val = dset.train_test_split(test_size=0.1)
ddict_with_val['validation'] = ddict_with_val['test']
ddict_with_val['train'] = ddict_with_val['train'].to_iterable_dataset()
del ddict_with_val['test']

tdc = TextDataControllerStreaming(ddict_with_val,
                                 main_text='Review Text',
                                 label_names=['Division Name','Department Name'],
                                 class_names_predefined=[['General', 'General Petite', 'Initmates'],
                                                         ['Bottoms', 'Dresses', 'Intimate', 'Jackets', 'Tops', 'Trend']],
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 metadatas=['Title','Age'],
                                 content_transformations=[text_normalize,str.lower],
                                 content_augmentations=contextual_aug_func,
                                 process_metas=True,
                                 batch_size=1000,
                                 num_proc=1, # remember to set num_proc to 1 whenever there's a GPU-based augmentation
                                 seed=42
                                )
tdc.process_and_tokenize(tokenizer,max_length=512)

-------------------- Data Filtering --------------------
Done
----- Metadata Simple Processing & Concatenating to Main Content -----
Done
----- Label Encoding -----


Map:   0%|          | 0/2253 [00:00<?, ? examples/s]

Done
-------------------- Dropping unused features --------------------
Done
----- Performing content transformation and tokenization on validation set -----


Map:   0%|          | 0/2253 [00:00<?, ? examples/s]

Map:   0%|          | 0/2253 [00:00<?, ? examples/s]

Map:   0%|          | 0/2253 [00:00<?, ? examples/s]

Done
----- Creating a generator for content transformation, augmentation and tokenization on train set -----
Done


In [ ]:
for i,v in enumerate(tdc.main_ddict['train']):
    if i==10:break
    print(f"Text: {v['Review Text']}\nLabel: {v['Division Name'],v['Department Name']} => {v['label']}")
    print('-'*10)

Text: 63 . beautiful top , worth the necessary tailoring . the beautiful bold print drew me to this top and it did not disappoint upon receipt . however , the bottom ruffle belled so far out on each side that it was laughable ! the actual fit is nothing like the picture ; clearly the model's arms are placed in front of all the extra fabric to hold the ruffle back . however , the fabric is beautiful , the fit was perfect ( size 2 , 5 ' 4 " , 106 lbs . ) , the quality is great and i love the print so i decided to take it to my tailor to " sew away " the " wings " on both si
Label: ('General Petite', 'Tops') => [1, 4]
----------
Text: 39. not as short on me ( petite ). i hate the small p as this one is not a fitted dress, and that was the same size for you. only thing. the length is a bit linger still 9 lower on calf for me ), the straps are almost tight, so i would say the dress is a reversed taper shape. this is beautiful, i ordered green as the other color ( plum ) doesn't have petite 

In [ ]:
for i in range(5):
    print(f"Text: {tdc.main_ddict['validation']['Review Text'][i]}")
    print(f"Label: {tdc.main_ddict['validation']['Division Name'][i],tdc.main_ddict['validation']['Department Name'][i]} => {tdc.main_ddict['validation']['label'][i]}")
    print('-'*10)

Text: 46 . soft , feminine and fun pockets ! . i love this tunic . purchased the dark orange in medium ( i am 5 ' 9 and 140 lbs ) . tried the small and almost kept it but i felt seams around my arm pits a tad , so went with the medium and glad i did - this top should be comfortable . feels very fall and perfect for casual get-togethers and running around town . only comment is that it is rayon ... and for me anyway rayon doesn't wash too well - so we shall see how this one fairs .
Label: ('General', 'Tops') => [0, 4]
----------
Text: 55 . a new staple ! . tried these on out of sheer curiosity -- i've got a long torso & was pleasantly surprised how flattering they are ! they manage to look flowing & sleek without shortening the legs . took a size 6 with my 27 " waist , 37 " hips . it's a bit of a generous fit , especially around the waist , but they're extremely comfortable & have room to tuck tops into . i have the cowled sweater tank in gray & it looks fantastic over these ! couldn't 

## Save and Load TextDataControllerStreaming

In [ ]:
show_doc(TextDataControllerStreaming.save_as_pickles)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L746){target="_blank" style="float:right; font-size:smaller"}

### TextDataControllerStreaming.save_as_pickles

>      TextDataControllerStreaming.save_as_pickles (fname,
>                                                   parent='pickle_files',
>                                                   drop_attributes=False)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| fname |  |  | Name of the pickle file |
| parent | str | pickle_files | Parent folder |
| drop_attributes | bool | False | Whether to drop large-size attributes |

In [ ]:
show_doc(TextDataControllerStreaming.from_pickle)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L724){target="_blank" style="float:right; font-size:smaller"}

### TextDataControllerStreaming.from_pickle

>      TextDataControllerStreaming.from_pickle (fname, parent='pickle_files')

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| fname |  |  | Name of the pickle file |
| parent | str | pickle_files | Parent folder |

TextDataControllerStreaming object can be saved and loaded with ease. This is especially useful after text processing and/or tokenization have been done

In [ ]:
from datasets import disable_caching

In [ ]:
disable_caching() # disable huggingface caching to see data size

In [ ]:
from underthesea import text_normalize
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw

In [ ]:
def nlp_aug_stochastic(x,aug=None,p=0.5):
    if not isinstance(x,list): 
        if random.random()<p: return aug.augment(x)[0]
        return x
    news=[]
    originals=[]
    for _x in x:
        if random.random()<p: news.append(_x)
        else: originals.append(_x)
    # only perform augmentation when needed
    if len(news): news = aug.augment(news)
    return news+originals

In [ ]:
aug2 = naw.ContextualWordEmbsAug(model_path='roberta-base', 
                                device='cuda:0', # if you don't have gpu, change to 'cpu'
                                action="substitute",
                                top_k=10,
                               aug_p=0.07)

contextual_aug_func = partial(nlp_aug_stochastic,aug=aug2,p=0.1)
# add these 2 instance variables to your gpu augmentation
# contextual_aug_func.run_on_gpu=True
# contextual_aug_func.batch_size=32

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
ddict_with_val = dset.train_test_split(test_size=0.2)
ddict_with_val['validation'] = ddict_with_val['test']
ddict_with_val['train'] = ddict_with_val['train'].to_iterable_dataset()
del ddict_with_val['test']

tdc = TextDataControllerStreaming(ddict_with_val,
                                 main_text='Review Text',
                                 label_names='Department Name',
                                 class_names_predefined=['Bottoms', 'Dresses', 'Intimate', 'Jackets', 'Tops', 'Trend'],
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 metadatas=['Title','Division Name'],
                                 content_transformations=[text_normalize,str.lower],
                                 content_augmentations= contextual_aug_func,
                                 process_metas=True,
                                 batch_size=100,
                                 num_proc=1, # remember to set num_proc to 1 whenever there's a GPU-based augmentation
                                 seed=42
                                )
tdc.process_and_tokenize(tokenizer,max_length=512)

-------------------- Data Filtering --------------------


Filter:   0%|          | 0/4698 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4530 [00:00<?, ? examples/s]

Done
----- Metadata Simple Processing & Concatenating to Main Content -----


Map:   0%|          | 0/4529 [00:00<?, ? examples/s]

...

Done
-------------------- Dropping unused features --------------------
Done
----- Performing content transformation and tokenization on validation set -----


Map:   0%|          | 0/4529 [00:00<?, ? examples/s]

Map:   0%|          | 0/4529 [00:00<?, ? examples/s]

Map:   0%|          | 0/4529 [00:00<?, ? examples/s]

Done
----- Creating a generator for content transformation, augmentation and tokenization on train set -----
Done


In [ ]:
tdc.main_ddict

DatasetDict({
    train: <datasets.iterable_dataset.IterableDataset object>
    validation: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4529
    })
})

In [ ]:
tdc.save_as_pickles('my_tdc_stream')

Let's check the file size

In [ ]:
file_stats = os.stat(Path('pickle_files/my_tdc_stream.pkl'))
print(f'File Size in MegaBytes is {round(file_stats.st_size / (1024 * 1024), 3)}')

File Size in MegaBytes is 479.179


Load back our object

In [ ]:
tdc2 = TextDataController.from_pickle('my_tdc_stream')

You can still access all its attributes, data, preprocessings, transformation/augmentation ...

In [ ]:
tdc2.main_ddict

DatasetDict({
    train: <datasets.iterable_dataset.IterableDataset object>
    validation: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4529
    })
})

In [ ]:
for i,v in enumerate(tdc2.main_ddict['train']):
    if i==3:break
    print(f"Text: {v['Review Text']}\nLabel: {v['Department Name']} => {v['label']}")
    print('-'*10)

Text: general . eye spy a great vest . i purchased this in my usual small ( size 4-6 ) and it fits just the way it shows in the photo . it is very flowy which is th point . i wore it over a black romper and it looked great . i can also wear with jeans and a simple black tank . keep the vest open or use the hook and eye and close it up . seeing the black through the sheer white is simply dreamy ! glad i purchased it during the sale on sale promotion . it is a classic piece for sure .
Label: Jackets => 3
----------
Text: general petite.. i love this soft, colorful, flowy beauty! it's the new color palette! i'm 5'5 ", 34 d, size 6 and a small fit me with room to spare. don't wait!
Label: Dresses => 1
----------
Text: general petite . cool top . impecable workmanship ( overseas ) . i usually wear a petite 2 but ordered this in a regular size 0 and glad that i did . since it curves up on this side , it barely overs the waistband on my jeans . ordered this in pink but it's more of pale coral

In [ ]:
tdc2.label_lists

[['Bottoms', 'Dresses', 'Intimate', 'Jackets', 'Tops', 'Trend']]

In [ ]:
tdc2.filter_dict,tdc2.content_tfms,tdc2.aug_tfms

({'Review Text': <function __main__.<lambda>(x)>,
  'Department Name': <function __main__.<lambda>(x)>},
 [<function underthesea.pipeline.text_normalize.text_normalize(text, tokenizer='underthesea')>,
  <method 'lower' of 'str' objects>],
 [functools.partial(<function nlp_aug_stochastic>, aug=<nlpaug.augmenter.word.context_word_embs.ContextualWordEmbsAug object>, p=0.1)])

If you don't want to store the HuggingFace DatasetDict in your `TextDataControllerStreaming`, or the augmentation functions (typically when you already have a trained model, and you only use `TextDataControllerStreaming` to preprocess the test set), you can remove it in the `save_as_pickles` step

In [ ]:
tdc.save_as_pickles('my_lightweight_tdc_stream',drop_attributes=True)

Let's check the file size

In [ ]:
file_stats = os.stat(Path('pickle_files/my_lightweight_tdc_stream.pkl'))
print(f'File Size in MegaBytes is {round(file_stats.st_size / (1024 * 1024), 3)}')

File Size in MegaBytes is 2.069


Load it back

In [ ]:
tdc3 = TextDataController.from_pickle('my_lightweight_tdc_stream')

We will use this object to demonstrate the Test Set Construction in the next section

### Construct a Test Dataset

In [ ]:
show_doc(TextDataControllerStreaming.prepare_test_dataset)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L970){target="_blank" style="float:right; font-size:smaller"}

### TextDataControllerStreaming.prepare_test_dataset

>      TextDataControllerStreaming.prepare_test_dataset (test_dset,
>                                                        do_filtering=False)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| test_dset |  |  | The HuggingFace Dataset as Test set |
| do_filtering | bool | False | whether to perform data filtering on this test set |

In [ ]:
show_doc(TextDataControllerStreaming.prepare_test_dataset_from_csv)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L937){target="_blank" style="float:right; font-size:smaller"}

### TextDataControllerStreaming.prepare_test_dataset_from_csv

>      TextDataControllerStreaming.prepare_test_dataset_from_csv (file_path,
>                                                                 do_filtering=F
>                                                                 alse)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| file_path |  |  | path to csv file |
| do_filtering | bool | False | whether to perform data filtering on this test set |

In [ ]:
show_doc(TextDataControllerStreaming.prepare_test_dataset_from_df)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L947){target="_blank" style="float:right; font-size:smaller"}

### TextDataControllerStreaming.prepare_test_dataset_from_df

>      TextDataControllerStreaming.prepare_test_dataset_from_df (df,
>                                                                validate=True, 
>                                                                do_filtering=Fa
>                                                                lse)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df |  |  | Pandas Dataframe |
| validate | bool | True | whether to perform input data validation |
| do_filtering | bool | False | whether to perform data filtering on this test set |

In [ ]:
show_doc(TextDataControllerStreaming.prepare_test_dataset_from_raws)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L957){target="_blank" style="float:right; font-size:smaller"}

### TextDataControllerStreaming.prepare_test_dataset_from_raws

>      TextDataControllerStreaming.prepare_test_dataset_from_raws (content)

|    | **Details** |
| -- | ----------- |
| content | Either a single sentence, list of sentence or a dictionary with keys are metadata columns and values are list |

Let's say you have done your preprocessing and tokenization in your training set, and have a nicely trained model, ready to do inference on new data. Here is how you can use `TextDataControllerStreaming` to apply all the necessary preprocessings to your new data

We will reuse the lightweight tdc object we created in the previous section (since we don't really need all the training data just to construct new data). Also, we will take a small sample of our training data and pretend it is our test data

In [ ]:
tdc = TextDataController.from_pickle('my_lightweight_tdc_stream')

In [ ]:
df_test = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig').sample(frac=0.2,random_state=1)
# drop NaN values in the label column
df_test = df_test[~df_test['Department Name'].isna()].reset_index(drop=True)
df_test.shape

(4692, 10)

In [ ]:
df_test.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,872,42,Perfect for work and play,This shirt works for both going out and going ...,5,1,0,General,Tops,Knits
1,1033,40,NaN,I don't know why i had the opposite problem mo...,4,1,0,General Petite,Bottoms,Jeans
2,1037,45,Great pants,These cords are great--lightweight for fl wint...,5,1,1,General Petite,Bottoms,Jeans
3,829,35,Surprisingly comfy for a button down,I am a 10 m and got the 10. it fits perfectly ...,5,1,1,General Petite,Tops,Blouses
4,872,29,Short and small,The shirt is mostly a thick sweatshirt materia...,3,0,15,General Petite,Tops,Knits


In [ ]:
test_dset = tdc.prepare_test_dataset_from_df(df_test,validate=True,do_filtering=True)

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title          758
Review Text    164
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 2 rows
-------------------- Start Test Set Transformation --------------------
-------------------- Data Filtering --------------------


Filter:   0%|          | 0/4692 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4528 [00:00<?, ? examples/s]

Done
----- Metadata Simple Processing & Concatenating to Main Content -----


Map:   0%|          | 0/4528 [00:00<?, ? examples/s]

Done
-------------------- Dropping unused features --------------------
Done
----- Performing content transformation and tokenization on test set -----


Map:   0%|          | 0/4528 [00:00<?, ? examples/s]

Map:   0%|          | 0/4528 [00:00<?, ? examples/s]

Map:   0%|          | 0/4528 [00:00<?, ? examples/s]

Done


In [ ]:
for i in range(3):
    print(f"Text: {test_dset['Review Text'][i]}")
    print(f"Input_ids: {test_dset['input_ids'][i]}")
    print('-'*10)

Text: general . perfect for work and play . this shirt works for both going out and going to work , and i can wear it with everything . fits perfect , tucked and untucked , tied and untied . i love it .
Input_ids: [0, 15841, 479, 1969, 13, 173, 8, 310, 479, 42, 6399, 1364, 13, 258, 164, 66, 8, 164, 7, 173, 2156, 8, 939, 64, 3568, 24, 19, 960, 479, 10698, 1969, 2156, 21222, 8, 7587, 23289, 2156, 3016, 8, 7587, 2550, 479, 939, 657, 24, 479, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
----------
Text: general petite . . i don't know why i had the opposite problem most reviewers had with these ..... i tried on the regular length in the store and found that they were just a bit too short with heels . ( i'm 5 ' 5 ) . i had them ordere

Let's make our test data streamed as well

In [ ]:
test_dset_raw = Dataset.from_pandas(df_test).to_iterable_dataset()

This test dataset might have some NaN values in the text field (`Review Text`), thus we will turn on the filtering option to get rid of these NaNs, as this is what we did in the training set. If your test dataset don't need any filtering, turn off this option

In [ ]:
test_dset = tdc.prepare_test_dataset(test_dset_raw,do_filtering=True)

-------------------- Start Test Set Transformation --------------------
-------------------- Data Filtering --------------------
Done
----- Metadata Simple Processing & Concatenating to Main Content -----
Done
-------------------- Dropping unused features --------------------
Done
----- Performing content transformation and tokenization on test set -----
Done


In [ ]:
for i,v in enumerate(test_dset):
    if i==3:break
    print(f"Text: {v['Review Text']}\Input_ids: {v['input_ids']}\nAttention mask: {v['attention_mask']}")
    print('-'*10)

Text: general . perfect for work and play . this shirt works for both going out and going to work , and i can wear it with everything . fits perfect , tucked and untucked , tied and untied . i love it .\Input_ids: [0, 15841, 479, 1969, 13, 173, 8, 310, 479, 42, 6399, 1364, 13, 258, 164, 66, 8, 164, 7, 173, 2156, 8, 939, 64, 3568, 24, 19, 960, 479, 10698, 1969, 2156, 21222, 8, 7587, 23289, 2156, 3016, 8, 7587, 2550, 479, 939, 657, 24, 479, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Attention mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
# #| hide
# import nbdev; nbdev.nbdev_export()